# CALCULATING  CRUDE DEATH RATE AND THE AGE-STANDARDIZED DEATH  RATE FOR BOTH UGANDA AND USA IN 2019.

**Crude death rate**-the number of deaths occurring during the year, per  100,000 population estimated at midyear\
**Age standardized death rate** -is the death rate of a population adjusted to a standard age distribution.

This notebook has two parts.
- Part 1- Data retrieval and Data wrangling
- Part 2- Script Writing

### **Data Retrieval and Data Wrangling**
The sources of my data  are:
- UN World Population Prospects (2022) — Population Estimates 1950-2021- here I retrieved Uganda Population,USA Population and the World population of 2019 with the 5 year age group.
- The table that you gave me for the interview- Table of age-specific death rates of COPD
- WHO Standard Population — Table 1 in 'Ahmad OB, Boschi-Pinto C, Lopez AD, Murray CJ, Lozano R, Inoue M (2001). Age standardization of rates: a new WHO standard.

I imported the three datasets from the UN database and  I created a Pandas DataFrame manually for the table you gave me plus the WHO standard population.

**Data Cleaning**.

I wanted all the age groups to be similar and resemble the table that you gave me. I removed some rows, created masks and unpivoted the World Population table. After that I aggregated all the ages from 85 years and above to be one. We shall see how I did it in the notebook.

In the Uganda and USA tables, I dropped some columns and aggregated all the ages from 85 years and above.

After the cleaning, I merged the five datasets  and created some new columns which helped me to calculate the crude rate and the age-standardized death rate.

**Data Scripting**

In this part I wrote an automation script. In this script I entered the formulas to calculate the crude death rate and age-standardized death rate for both Uganda and USA




In [1]:
# pandas is used for data wrangling.
import pandas as pd

In [2]:
# The world population data
world_population=pd.read_excel("WPP2022_POP_F02_1_POPULATION_5-YEAR_AGE_GROUPS_BOTH_SEXES.xlsx")

In [3]:
# I prefer preserving the original data. This is why I usually copy and use the copied dataset.
world_copy=world_population.copy()

In [ ]:
# The world dataset column has Nans rows upto the 15th row. The 15 th row also contains the column names
# so I will first set the dataset to start from the 15 th index using the iloc. arument


In [5]:
world_copy=world_copy.iloc[15:]

In [6]:
# I will then make the first row in the filtered dataset to be the column and then delete it to eliminate redudancy.
world_copy.columns=world_copy.iloc[0]
world_copy = world_copy[1:]

In [7]:
# here I create a mask that only filters the dataset around the whole world and the year 2019.
world_copy=world_copy[(world_copy['Region, subregion, country or area *']=='WORLD') & (world_copy['Year']==2019)]

In [8]:
# As we can see the data is in the wide format. it is important to change it to a long format. It will make analysis easy
world_copy

15,Index,Variant,"Region, subregion, country or area *",Notes,Location code,ISO3 Alpha-code,ISO2 Alpha-code,SDMX code**,Type,Parent code,...,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90-94,95-99,100+
85,70,Estimates,WORLD,NaN,900,NaN,NaN,1,World,0,...,372483.558,319586.541,260373.821,182094.5855,124404.171,83165.392,43535.115,16657.5595,3917.9225,503.572


In [9]:
# I first drop some columns and remain with the only the age group column
drop_columns=['Index', 'Variant', 'Region, subregion, country or area *', 'Notes',
       'Location code', 'ISO3 Alpha-code', 'ISO2 Alpha-code', 'SDMX code**',
       'Type', 'Parent code', 'Year']

In [10]:
world_copy.drop(columns=drop_columns, inplace=True)

In [11]:
world_copy.head()

15,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,...,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90-94,95-99,100+
85,684872.621,678417.166,645915.768,615536.005,596534.013,603855.871,592985.944,540438.207,485895.6165,477194.9815,...,372483.558,319586.541,260373.821,182094.5855,124404.171,83165.392,43535.115,16657.5595,3917.9225,503.572


In [12]:
# I then unpivot the wide table to long table and do change the column names
world_table= pd.melt(world_copy) 
world_table.rename(columns={15: 'Age Group', 'value': 'Population (Thousands)'}, inplace=True)

In [13]:
world_table

,Age Group,Population (Thousands)
0,0-4,684872.621
1,5-9,678417.166
2,10-14,645915.768
3,15-19,615536.005
4,20-24,596534.013
5,25-29,603855.871
6,30-34,592985.944
7,35-39,540438.207
8,40-44,485895.6165
9,45-49,477194.9815


In [14]:
# death rates - import the death rates dataset by making a dataframe
death_rates_data= {
    
    'Age Group': ['0-4', '5-9', '10-14', '15-19', '20-24','25-29','30-34',
                  '35-39','40-44','45-49','50-54','55-59','60-64','65-69',
                  '70-74','75-79','80-84','85+'
                 ],
    'USA Death Rate': [0.04, 0.02, 0.02, 0.02, 0.06, 0.11, 0.29, 0.56, 1.42, 4.00,
                      14.13, 37.22, 66.48, 108.66, 213.10, 333.06, 491.10, 894.45],
    'UGANDA Death Rate': [0.40, 0.17, 0.07, 0.23, 0.38, 0.40, 0.75, 1.11, 2.04,
                         5.51, 13.26, 33.25, 69.62, 120.78, 229.88, 341.06, 529.31, 710.40]
}

death_rates = pd.DataFrame(death_rates_data)
death_rates

,Age Group,USA Death Rate,UGANDA Death Rate
0,0-4,0.04,0.40
1,5-9,0.02,0.17
2,10-14,0.02,0.07
3,15-19,0.02,0.23
4,20-24,0.06,0.38
5,25-29,0.11,0.40
6,30-34,0.29,0.75
7,35-39,0.56,1.11
8,40-44,1.42,2.04
9,45-49,4.00,5.51


In [15]:
# standardized population- import the standardized population by making a dataframe
data_1= {
    
    'Age Group': ['0-4', '5-9', '10-14', '15-19', '20-24','25-29','30-34',
                  '35-39','40-44','45-49','50-54','55-59','60-64','65-69',
                  '70-74','75-79','80-84','85+'
                 ],
    'Std World Avearage (%)': [8.86, 8.69, 8.60, 8.47, 8.22, 7.93, 7.61, 7.15,
                               6.59, 6.04, 5.37, 4.55, 3.72, 2.96, 2.21, 1.52,
                               0.91, 0.63]
}
standardized_population=pd.DataFrame(data_1)
standardized_population

,Age Group,Std World Avearage (%)
0,0-4,8.86
1,5-9,8.69
2,10-14,8.60
3,15-19,8.47
4,20-24,8.22
5,25-29,7.93
6,30-34,7.61
7,35-39,7.15
8,40-44,6.59
9,45-49,6.04


In [16]:
# import the csv files- uganda and usa- the datasets that I use for uganda and usa population
uganda=pd.read_csv("unpopulation_dataportal_20240226172508.csv")
usa=pd.read_csv("unpopulation_dataportal_20240226174051.csv")

In [17]:
uganda_copy=uganda.copy()

In [18]:
uganda_copy.head()

,IndicatorId,IndicatorName,IndicatorShortName,Source,SourceYear,Author,LocationId,Location,Iso2,Iso3,...,AgeStart,AgeEnd,Age,CategoryId,Category,EstimateTypeId,EstimateType,EstimateMethodId,EstimateMethod,Value
0,46,Population by 5-year age groups and sex,Annual population by 5-year age groups and by sex,World Population Prospects,2022,United Nations Population Division,800,Uganda,UG,UGA,...,0,4.0,0-4,0,Not applicable,1,Model-based Estimates,2,Interpolation,7328968.0
1,46,Population by 5-year age groups and sex,Annual population by 5-year age groups and by sex,World Population Prospects,2022,United Nations Population Division,800,Uganda,UG,UGA,...,5,9.0,5-9,0,Not applicable,1,Model-based Estimates,2,Interpolation,6614420.5
2,46,Population by 5-year age groups and sex,Annual population by 5-year age groups and by sex,World Population Prospects,2022,United Nations Population Division,800,Uganda,UG,UGA,...,10,14.0,10-14,0,Not applicable,1,Model-based Estimates,2,Interpolation,5899400.5
3,46,Population by 5-year age groups and sex,Annual population by 5-year age groups and by sex,World Population Prospects,2022,United Nations Population Division,800,Uganda,UG,UGA,...,15,19.0,15-19,0,Not applicable,1,Model-based Estimates,2,Interpolation,5151082.0
4,46,Population by 5-year age groups and sex,Annual population by 5-year age groups and by sex,World Population Prospects,2022,United Nations Population Division,800,Uganda,UG,UGA,...,20,24.0,20-24,0,Not applicable,1,Model-based Estimates,2,Interpolation,4348173.0


In [19]:
# I drop all other columns and remain only with the age group and the total population
columns_drop=['IndicatorId', 'IndicatorName', 'IndicatorShortName', 'Source',
       'SourceYear', 'Author', 'LocationId', 'Location', 'Iso2', 'Iso3',
       'TimeId', 'Time', 'VariantId', 'Variant', 'SexId', 'Sex', 'AgeId',
       'AgeStart', 'AgeEnd', 'CategoryId', 'Category', 'EstimateTypeId',
       'EstimateType', 'EstimateMethodId', 'EstimateMethod']

In [20]:
uganda_copy.drop(columns=columns_drop, inplace=True)

In [21]:
uganda_copy

,Age,Value
0,0-4,7328968.0
1,5-9,6614420.5
2,10-14,5899400.5
3,15-19,5151082.0
4,20-24,4348173.0
5,25-29,3499504.0
6,30-34,2618559.0
7,35-39,1903175.0
8,40-44,1503669.5
9,45-49,1234979.0


In [22]:
uganda_copy.rename(columns={'Value': 'Population (Uganda)'},inplace=True)

In [23]:
usa_copy=usa.copy()

In [24]:
# I drop all other columns and remain with the age group and the USA population
dropped_columns=['IndicatorId', 'IndicatorName', 'IndicatorShortName', 'Source',
       'SourceYear', 'Author', 'LocationId', 'Location', 'Iso2', 'Iso3',
       'TimeId', 'Time', 'VariantId', 'Variant', 'SexId', 'Sex', 'AgeId',
       'AgeStart', 'AgeEnd', 'CategoryId', 'Category', 'EstimateTypeId',
       'EstimateType', 'EstimateMethodId', 'EstimateMethod']

In [25]:
usa_copy.drop(columns=dropped_columns,inplace=True)

In [26]:
usa_copy.rename(columns={'Value': 'Population (USA)'},inplace=True)

In [27]:
usa_copy

,Age,Population (USA)
0,0-4,19848556.0
1,5-9,20697075.5
2,10-14,22092167.5
3,15-19,21895122.5
4,20-24,21871808.5
5,25-29,23406927.5
6,30-34,22842151.5
7,35-39,22296952.5
8,40-44,20694555.5
9,45-49,21244258.5


In [28]:
# As we can see in the USA filtered datset. the age groups have reached to 100+ but I need the final entry to be 85+
# I add the three columns, then drop them and finally append the three added columns
res1 = usa_copy.iloc[[17, 18,19,20], :].sum()
usa_copy = usa_copy.drop([17, 18, 19, 20], axis=0).append(res1, ignore_index=True)


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2200\2090830333.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  usa_copy = usa_copy.drop([17, 18, 19, 20], axis=0).append(res1, ignore_index=True)


In [29]:
usa_copy["Age"] = usa_copy["Age"].replace("85-8990-9495-99100+", "85+")

In [30]:
usa_copy

,Age,Population (USA)
0,0-4,19848556.0
1,5-9,20697075.5
2,10-14,22092167.5
3,15-19,21895122.5
4,20-24,21871808.5
5,25-29,23406927.5
6,30-34,22842151.5
7,35-39,22296952.5
8,40-44,20694555.5
9,45-49,21244258.5


In [31]:
# As we can see in the UGANDA filtered datset. the age groups have reached to 100+ but I need the final entry to be 85+
# I add the three columns, then drop them and finally append the three added columns
res2 = uganda_copy.iloc[[17, 18,19,20], :].sum()
uganda_copy = uganda_copy.drop([17, 18, 19, 20], axis=0).append(res2, ignore_index=True)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2200\330230308.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  uganda_copy = uganda_copy.drop([17, 18, 19, 20], axis=0).append(res2, ignore_index=True)


In [32]:
uganda_copy["Age"] = uganda_copy["Age"].replace("85-8990-9495-99100+", "85+")

In [33]:
uganda_copy

,Age,Population (Uganda)
0,0-4,7328968.0
1,5-9,6614420.5
2,10-14,5899400.5
3,15-19,5151082.0
4,20-24,4348173.0
5,25-29,3499504.0
6,30-34,2618559.0
7,35-39,1903175.0
8,40-44,1503669.5
9,45-49,1234979.0


In [34]:
world_table_copy=world_table.copy()

In [35]:
# I also do the same to the world dataset
world_table_copy

,Age Group,Population (Thousands)
0,0-4,684872.621
1,5-9,678417.166
2,10-14,645915.768
3,15-19,615536.005
4,20-24,596534.013
5,25-29,603855.871
6,30-34,592985.944
7,35-39,540438.207
8,40-44,485895.6165
9,45-49,477194.9815


In [36]:
res3 = world_table_copy.iloc[[17, 18,19,20], :].sum()
world_table_copy= world_table_copy.drop([17, 18, 19, 20], axis=0).append(res3, ignore_index=True)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2200\4285685761.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  world_table_copy= world_table_copy.drop([17, 18, 19, 20], axis=0).append(res3, ignore_index=True)


In [37]:
world_table_copy["Age Group"] = world_table_copy["Age Group"].replace("85-8990-9495-99100+", "85+")

In [39]:
world_table_copy.rename(columns={'Population (Thousands)': ' World Population (Thousands)'},inplace=True)

In [41]:
world_table_copy

,Age Group,World Population (Thousands)
0,0-4,684872.621
1,5-9,678417.166
2,10-14,645915.768
3,15-19,615536.005
4,20-24,596534.013
5,25-29,603855.871
6,30-34,592985.944
7,35-39,540438.207
8,40-44,485895.6165
9,45-49,477194.9815


In [42]:
# I merge the 5 datasets and come up with one table that will assist me to calculate the crude death rate and the age standardized death rate
merged_df = pd.merge(death_rates, standardized_population, on='Age Group', how='outer')
merged_df = pd.merge(merged_df,world_table_copy , on='Age Group', how='outer')


In [43]:
merged_df.head()

,Age Group,USA Death Rate,UGANDA Death Rate,Std World Avearage (%),World Population (Thousands)
0,0-4,0.04,0.40,8.86,684872.621
1,5-9,0.02,0.17,8.69,678417.166
2,10-14,0.02,0.07,8.60,645915.768
3,15-19,0.02,0.23,8.47,615536.005
4,20-24,0.06,0.38,8.22,596534.013


In [44]:
merged_df_2 = pd.merge(usa_copy, uganda_copy, on='Age', how='outer')

In [45]:
merged_df_2.head()

,Age,Population (USA),Population (Uganda)
0,0-4,19848556.0,7328968.0
1,5-9,20697075.5,6614420.5
2,10-14,22092167.5,5899400.5
3,15-19,21895122.5,5151082.0
4,20-24,21871808.5,4348173.0


In [46]:
final_df = pd.concat([merged_df, merged_df_2], axis=1)

In [47]:
final_df.drop(columns='Age',inplace=True)

In [48]:
# Reorder columns

final_df = final_df[['Age Group','USA Death Rate','UGANDA Death Rate','Population (USA)','Population (Uganda)',' World Population (Thousands)','Std World Avearage (%)']]            

In [49]:
final_df.head()

,Age Group,USA Death Rate,UGANDA Death Rate,Population (USA),Population (Uganda),World Population (Thousands),Std World Avearage (%)
0,0-4,0.04,0.40,19848556.0,7328968.0,684872.621,8.86
1,5-9,0.02,0.17,20697075.5,6614420.5,678417.166,8.69
2,10-14,0.02,0.07,22092167.5,5899400.5,645915.768,8.60
3,15-19,0.02,0.23,21895122.5,5151082.0,615536.005,8.47
4,20-24,0.06,0.38,21871808.5,4348173.0,596534.013,8.22


In [50]:
final_copy=final_df.copy()

In [52]:
# let me start some feature engineering

# I will  add USA DEATHS,UGANDA DEATHS,STANDARD POPULATION,UGANDA EXPECTED DEATHS,USA EXPECTED DEATHS
'''The following are the formulas that I will use
    USA DEATHS=(USA DEATH RATE*USA POPULATION)/100000
    UGANDA DEATHS=(UGANDA DEATH RATE*UGANDA POPULATION)/100000
    STANDARD POPULATION=World Population*std_world average
    USA EXPECTED DEATH=(USA DEATH RATE * STD POPULATION)/100000
    UGANDA EXPECTED DEATH=(UGANDA DEATH RATE * STD POPULATION)/100000
    
    '''

final_copy['USA Deaths']=(final_copy['USA Death Rate'] *final_copy['Population (USA)'])/100000
final_copy['UGANDA Deaths']=(final_copy['UGANDA Death Rate'] *final_copy['Population (Uganda)'])/100000
final_copy['Standard Population']=(final_copy[' World Population (Thousands)']*1000)*(final_copy['Std World Avearage (%)']/100)
final_copy['USA Expected Death Rate']=(final_copy['USA Death Rate'] *final_copy['Standard Population'])/100000
final_copy['UGANDA Expected Death Rate']=(final_copy['UGANDA Death Rate'] *final_copy['Standard Population'])/100000


In [53]:
final_copy

,Age Group,USA Death Rate,UGANDA Death Rate,Population (USA),Population (Uganda),World Population (Thousands),Std World Avearage (%),USA Deaths,UGANDA Deaths,Standard Population,USA Expected Death Rate,UGANDA Expected Death Rate
0,0-4,0.04,0.40,19848556.0,7328968.0,684872.621,8.86,7.939422,29.315872,60679714.2206,24.271886,242.718857
1,5-9,0.02,0.17,20697075.5,6614420.5,678417.166,8.69,4.139415,11.244515,58954451.7254,11.79089,100.222568
2,10-14,0.02,0.07,22092167.5,5899400.5,645915.768,8.60,4.418434,4.129580,55548756.048,11.109751,38.884129
3,15-19,0.02,0.23,21895122.5,5151082.0,615536.005,8.47,4.379024,11.847489,52135899.6235,10.42718,119.912569
4,20-24,0.06,0.38,21871808.5,4348173.0,596534.013,8.22,13.123085,16.523057,49035095.8686,29.421058,186.333364
5,25-29,0.11,0.40,23406927.5,3499504.0,603855.871,7.93,25.747620,13.998016,47885770.5703,52.674348,191.543082
6,30-34,0.29,0.75,22842151.5,2618559.0,592985.944,7.61,66.242239,19.639193,45126230.3384,130.866068,338.446728
7,35-39,0.56,1.11,22296952.5,1903175.0,540438.207,7.15,124.862934,21.125242,38641331.8005,216.391458,428.918783
8,40-44,1.42,2.04,20694555.5,1503669.5,485895.6165,6.59,293.862688,30.674858,32020521.12735,454.6914,653.218631
9,45-49,4.00,5.51,21244258.5,1234979.0,477194.9815,6.04,849.770340,68.047343,28822576.8826,1152.903075,1588.123986


In [54]:
def script(dataframe):
    '''
    Uganda crude rate=(total(uganda deaths)/total(uganda population)*100000
    Usa crude rate=(total(usa deaths)/total(usa population)*100000
    uganda_std_death_rate=(total(ug_expected_death)/total(standard_pop)*100000)
    usa_std_death_rate=(total(usa_expected_death)/total(standard_pop)*100000)
    '''

    UG_crude_rate = round((dataframe['UGANDA Deaths'].sum()) / (dataframe['Population (Uganda)'].sum()) * 100000, 1)
    USA_crude_rate = round((dataframe['USA Deaths'].sum()) / (dataframe['Population (USA)'].sum()) * 100000, 1)
    Uganda_standard_death_rate = round((dataframe['UGANDA Expected Death Rate'].sum()) / (dataframe['Standard Population'].sum()) * 100000, 1)
    USA_standard_death_rate = round((dataframe['USA Expected Death Rate'].sum()) / (dataframe['Standard Population'].sum()) * 100000, 1)

    print(f"Uganda crude rate: {UG_crude_rate}")
    print(f"USA crude rate: {USA_crude_rate}")
    print(f"Uganda standard death rate: {Uganda_standard_death_rate}")
    print(f"USA standard death rate: {USA_standard_death_rate}")


script(final_copy)


Uganda crude rate: 5.8
USA crude rate: 57.2
Uganda standard death rate: 9.9
USA standard death rate: 9.4


The crud death rate of US is higher than that of Uganda since US has a higher population compared to Uganda. However, when we use a standardized age we see that the death rateof Uganda is higher than that of US.